# Aplicación del método Bouyoucus

Primero, ejecutamos las siguientes celdas, que contienen las funciones para ajustar a la función logaritmica de diámetro sobre el porcentaje de peso.

In [2]:
import numpy as np
import pandas as pd


def texture(clay, silt, sand):
    """Gets the type of texture from the givin percentage of clay, silt and sand."""
    thresholds = [
        ("arcillosa", [40.0, 100.0], [0.0, 60.0], [0.0, 65.0]),
        ("franca", [7.0, 27.0], [28.0, 50.0], [23.0, 52.0]), 
        ("arcillo arenosa", [35.0, 55.0], [0.0, 20.0], [45.0, 65.0]), 
        ("franco arcillosa", [27.0, 40.0], [15.0, 53.0], [20.0, 45.0]), 
        ("franco arcillo limosa", [27.0, 40.0], [40.0, 73.0], [0.0, 20.0]), 
        ("arcillo limosa", [40.0, 60.0], [40.0, 60.0], [0.0, 20.0]), 
        ("franco arcillo arenosa", [20.0, 35.0], [0.0, 28.0], [45.0, 80.0]), 
        ("franco limosa", [0.0, 27.0], [50.0, 80.0], [0.0, 50.0]), 
        ("limosa", [0.0, 12.0], [80.0, 100.0], [0.0, 20.0]), 
        ("franco arenosa", [0.0, 20.0], [0.0, 50.0], [50.0, 70.0]), 
        ("arena franca", [0.0, 15.0], [0.0, 30.0], [70.0, 86.0]), 
        ("arenosa", [0.0, 10.0], [0.0, 14.0], [86.0, 100.0])
    ]
    for threshold in thresholds:
        if clay > threshold[1][0] and clay < threshold[1][1] and silt > threshold[2][0] and silt < threshold[2][1] and sand > threshold[3][0] and sand < threshold[3][1]:
            return threshold[0]
    return "unknown"


def fit_by_sample(data):
    """Gets the terrain type percentage."""
    x = data["Diámetro corregido (µm)"].to_numpy()
    y = data["% Peso"].to_numpy()
    try:
        coeficients = np.polyfit(np.log(x), y, 1)
        clay = coeficients[0] * np.log(2.0) + coeficients[1]
        silt = coeficients[0] * np.log(50.0) + coeficients[1] - clay
        sand = 100.0 - sum([clay, silt])
    except np.linalg.LinAlgError:
        clay = silt = sand = np.nan
    return pd.Series({"clay": clay, "silt": silt, "sand": sand})


Leemos los datos desde el Excel, y aplicamos las transformaciones necesarias.

In [22]:
# Read data from Excel
df = pd.read_excel("./DATOS TFG.xlsx", sheet_name="Python")

# Calcultes derivative data
df.loc[df["Tiempo"] == ">8h", "Tiempo"] = 480
df["Tiempo"] = df["Tiempo"].astype('float32')
df["Lc"] = df["Lecturas"] - df["Lo"]
df["θ"] = -0.2761 * df["Lecturas"] + 50.957
df["Diámetro (µm)"] = df["θ"] / np.sqrt(df["Tiempo"])
df["Diámetro corregido (µm)"] = df["Diámetro (µm)"] * df["factor T"]
df["% Peso"] = ((df["Lecturas"] - df["Lo"]) / df["Peso"]) * 100.0
kinds = df.groupby("Muestra").apply(fit_by_sample)
kinds["texture"] = kinds.apply(lambda data: texture(data["clay"], data["silt"], data["sand"]), axis=1)

/Users/marcos/.local/share/virtualenvs/bouyocus-method-6Ts6NAIv/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [23]:
df.to_excel("datos.xlsx")
kinds.to_excel("texturas.xlsx")